In [1]:
import os
import polars as pl
import altair as alt

In [2]:
import warnings
from src.kristi_promin import kristi_promin
from src.me_to_neurazi import me_to_neurazi
from src.alt_friendly import alt_friendly

pl.Config.set_tbl_rows(150)
pl.Config.set_fmt_str_lengths(150)
pl.Config.set_tbl_width_chars(200)

alt.themes.register('irozhlas', kristi_promin)
alt.themes.enable('irozhlas')
warnings.filterwarnings('ignore')

In [3]:
df = pl.scan_parquet(
    "data/projevy.parquet"
).filter(
    pl.col('datum').dt.year() >= 1993
).filter(
    pl.col("komora_komplet").str.contains('Poslanecká') | pl.col('komora_komplet').str.contains('PČR, PS')
).with_columns(
    pl.col('mluvci_id').str.split('/sqw/detail.sqw?id=').alias('id')
).with_columns(
    pl.when(pl.col('id').len() > 2).then(pl.col('id').list.slice(1)).alias('id')
).explode(
    "id"
).with_columns(
    pl.col("text").str.count_matches(" ").alias("pocet_slov") # tady budou zlobit dvojité mezery, pohlídat při čištění
).sort(
    by="datum"
).collect()

In [4]:
df.filter(pl.col("obdobi") == 2021).select(pl.col("pocet_slov")).sum().item()

15589023

In [5]:
df.filter(pl.col("obdobi") == 2021).select(pl.col("pocet_slov")).sum().item() / (105 * 60 * 8 * 5)

61.86120238095238

## Skokani

In [7]:
import re

In [8]:
def ocisti(frejm):
    return frejm.with_columns(pl.col('text').str.replace_all(r'[^\w\s]','',).str.to_lowercase())

In [9]:
def zgrupni_a_pores_cetnost(frejm):

    frejm_pracovni = frejm.with_columns(pl.col('text').str.split(' ')).explode('text').group_by('text').len().sort(by='len',descending=True)
    celkem = frejm_pracovni.select(pl.col('len')).sum().item()
    
    return frejm_pracovni.with_columns((pl.col('len') / celkem).alias('podil'))

In [10]:
stara_snemovna = zgrupni_a_pores_cetnost(ocisti(df.filter(pl.col("obdobi") == 2017)))

In [11]:
stara_snemovna

text,len,podil
str,u32,f64
"""a""",396529,0.03013
"""to""",277500,0.021086
"""se""",253120,0.019233
"""je""",232138,0.017639
"""v""",220998,0.016792
"""že""",209647,0.01593
"""na""",187316,0.014233
"""o""",125001,0.009498
"""tak""",101117,0.007683


In [12]:
nova_snemovna = zgrupni_a_pores_cetnost(ocisti(df.filter(pl.col("obdobi") == 2021)))

In [13]:
porovnani = stara_snemovna.rename(
    {'podil':'podil_2017','len':'pocet_2017'}
).join(
    nova_snemovna.rename(
        {'podil':'podil_2021','len':'pocet_2021'}
    ), on='text', how='full'
).with_columns(
    (pl.col('podil_2021') / pl.col('podil_2017')).alias('rozdil')
)

In [14]:
porovnani

text,pocet_2017,podil_2017,text_right,pocet_2021,podil_2021,rozdil
str,u32,f64,str,u32,f64,f64
"""a""",396529,0.03013,"""a""",484414,0.03091,1.025874
"""to""",277500,0.021086,"""to""",373088,0.023806,1.129017
"""se""",253120,0.019233,"""se""",299082,0.019084,0.992238
"""je""",232138,0.017639,"""je""",271748,0.01734,0.983042
"""v""",220998,0.016792,"""v""",258102,0.016469,0.980743
"""že""",209647,0.01593,"""že""",244555,0.015605,0.97958
"""na""",187316,0.014233,"""na""",232179,0.014815,1.040879
"""o""",125001,0.009498,"""o""",134088,0.008556,0.9008
"""tak""",101117,0.007683,"""tak""",121183,0.007732,1.006398


In [15]:
hranice = 30
porovnani.filter((pl.col('pocet_2017') >= hranice) & (pl.col('pocet_2021') >= hranice)).sort(by='rozdil',descending=True)

text,pocet_2017,podil_2017,text_right,pocet_2021,podil_2021,rozdil
str,u32,f64,str,u32,f64,f64
"""zmáčkne""",36,0.000003,"""zmáčkne""",2121,0.000135,49.475516
"""2026""",35,0.000003,"""2026""",1125,0.000072,26.992098
"""ztišení""",39,0.000003,"""ztišení""",1051,0.000067,22.630298
"""2024""",148,0.000011,"""2024""",3644,0.000233,20.676109
"""ukrajině""",90,0.000007,"""ukrajině""",2214,0.000141,20.657952
"""zdražování""",33,0.000003,"""zdražování""",781,0.00005,19.874182
"""ukrajiny""",79,0.000006,"""ukrajiny""",1746,0.000111,18.55963
"""stiskne""",81,0.000006,"""stiskne""",1531,0.000098,15.87239
"""2025""",146,0.000011,"""2025""",2676,0.000171,15.391658


In [16]:
hranice = 100
porovnani.filter((pl.col('pocet_2017') >= hranice) & (pl.col('pocet_2021') >= hranice)).sort(by='rozdil',descending=True)

text,pocet_2017,podil_2017,text_right,pocet_2021,podil_2021,rozdil
str,u32,f64,str,u32,f64,f64
"""2024""",148,0.000011,"""2024""",3644,0.000233,20.676109
"""2025""",146,0.000011,"""2025""",2676,0.000171,15.391658
"""energií""",173,0.000013,"""energií""",3052,0.000195,14.814622
"""2023""",280,0.000021,"""2023""",4625,0.000295,13.870939
"""zvedne""",202,0.000015,"""zvedne""",2747,0.000175,11.419825
"""teďka""",103,0.000008,"""teďka""",1328,0.000085,10.827122
"""fialy""",253,0.000019,"""fialy""",2712,0.000173,9.001633
"""místopředsedkyně""",474,0.000036,"""místopředsedkyně""",5007,0.000319,8.870568
"""korespondenční""",275,0.000021,"""korespondenční""",2876,0.000184,8.782302


In [17]:
hranice = 100
porovnani.filter((pl.col('pocet_2017') >= hranice) & (pl.col('pocet_2021') >= hranice)).sort(by='rozdil',descending=False).with_columns(1 / pl.col("rozdil"))

text,pocet_2017,podil_2017,text_right,pocet_2021,podil_2021,rozdil,literal
str,u32,f64,str,u32,f64,f64,f64
"""klaus""",793,0.00006,"""klaus""",102,0.000007,0.108014,9.258078
"""faltýnek""",1047,0.00008,"""faltýnek""",155,0.00001,0.124319,8.043829
"""testy""",678,0.000052,"""testy""",111,0.000007,0.137482,7.273686
"""nesouhlasné""",2064,0.000157,"""nesouhlasné""",356,0.000023,0.144841,6.904107
"""okd""",684,0.000052,"""okd""",118,0.000008,0.14487,6.902747
"""zahájil""",3737,0.000284,"""zahájil""",670,0.000043,0.150558,6.641958
"""čssd""",1871,0.000142,"""čssd""",344,0.000022,0.154396,6.47684
"""bonusu""",688,0.000052,"""bonusu""",131,0.000008,0.159895,6.254102
"""ministryni""",2735,0.000208,"""ministryni""",542,0.000035,0.166416,6.009051


## Skokani od roku 1993

In [19]:
import simplemma

In [20]:
def prvnipad(slovo):
    if len(slovo) > 2:
        return simplemma.lemmatize(slovo, lang='cs')
    else:
        return None

In [21]:
stare_snemovny = zgrupni_a_pores_cetnost(
    ocisti(df.filter(pl.col("obdobi").is_between(1993,2020)))
).with_columns(
    pl.col("text").map_elements(prvnipad, return_dtype=pl.String).alias("lemma")
).group_by(
    "lemma"
).agg([
    pl.col("len").sum(),
    pl.col("podil").sum()
]).rename(
    {'len':'len_93_21','podil':'podil_93_21'}
)

In [22]:
snemovna_2021 = zgrupni_a_pores_cetnost(ocisti(df.filter(pl.col("obdobi") == 2021))).with_columns(
    pl.col("text").map_elements(prvnipad, return_dtype=pl.String).alias("lemma")
).group_by(
    "lemma"
).agg([
    pl.col("len").sum(),
    pl.col("podil").sum()
]).rename(
    {'len':'len_21_25','podil':'podil_21_25'}
)

In [23]:
stare_snemovny.sample(10)

lemma,len_93_21,podil_93_21
str,u32,f64
"""multidetektorového""",1,1.5465e-8
"""drietoma""",1,1.5465e-8
"""přešlali""",1,1.5465e-8
"""novokeynesiánce""",1,1.5465e-8
"""traktorů""",36,5.5675e-7
"""13495""",1,1.5465e-8
"""eurodepozit""",1,1.5465e-8
"""2751996""",1,1.5465e-8
"""došetřena""",6,9.2791e-8


In [24]:
len(stare_snemovny)

260469

In [25]:
porovnani_93_21 = stare_snemovny.join(
    snemovna_2021, how='full', on='lemma'
).with_columns(
    (pl.col('podil_21_25') / pl.col('podil_93_21')).alias('rozdil')
)

In [26]:
porovnani_93_21.sort(by='rozdil',descending=True)

lemma,len_93_21,podil_93_21,lemma_right,len_21_25,podil_21_25,rozdil
str,u32,f64,str,u32,f64,f64
"""projednávatelné""",1,1.5465e-8,null,null,null,null
"""zaúvěrováno""",1,1.5465e-8,null,null,null,null
"""fekálie""",1,1.5465e-8,null,null,null,null
"""rozmíšek""",1,1.5465e-8,null,null,null,null
"""60119136""",1,1.5465e-8,null,null,null,null
"""spadaným""",1,1.5465e-8,null,null,null,null
"""klasifikovaných""",39,6.0314e-7,null,null,null,null
"""theresou""",2,3.0930e-8,null,null,null,null
"""inhalovat""",9,1.3919e-7,null,null,null,null


In [27]:
df.filter(pl.col("text").str.contains("[Bb]itcoin")).sort(by='datum').head(10)

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,mluvci_jisty,id,pocet_slov
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,bool,str,u32
"""Poslanec Zbyněk Stanjura""","""2013ps_stenprot_033schuz_s033113.htm""","""/sqw/detail.sqw?id=5969""","""Děkuji za slovo. Chci říci, že dodržíme dohodu, že dneska do 19 hodin ukončíme druhé čtení obou bodů, protože je sloučená rozprava, takže já toho řekn…",1,"""ps""",2013,33,113,2015-10-07,"""Parlament České republiky, Poslanecká sněmovna 2013 – 2017""","""33. schůze""",true,true,true,"""5969""",763
"""Poslanec Adolf Beznoska""","""2013ps_stenprot_047schuz_s047158.htm""","""/sqw/detail.sqw?id=6181""","""Děkuji za slovo. Vážený pane předsedající, milé kolegyně, vážení kolegové, dovolte, abych se krátce vyjádřil k předložené materii. Tato implementační …",5,"""ps""",2013,47,158,2016-05-31,"""Parlament České republiky, Poslanecká sněmovna 2013 – 2017""","""47. schůze""",true,true,true,"""6181""",576
"""Předseda vlády ČR Andrej Babiš""","""2017ps_stenprot_019schuz_s019103.htm""","""https://www.vlada.cz/cz/clenove-vlady/premier/zivotopis/andrej-babis-162063/""","""Dobrý den. Já jsem čekal, že piráti nám něco ukážou. Vy jste měli tolik řečí o tom IT, jaký jste fantastický experti. Tak kromě toho těžení bitcoinů n…",5,"""ps""",2017,19,103,2018-09-13,"""Parlament České republiky, Poslanecká sněmovna 2017 – 2021""","""19. schůze""",true,true,true,null,648
"""Ministryně financí ČR Alena Schillerová""","""2017ps_stenprot_020schuz_s020013.htm""","""https://www.vlada.cz/cz/clenove-vlady/alena-schillerova-167019/""","""Děkuji za slovo, pane předsedající. Vážené dámy, vážení pánové, dovolte, abych stručně uvedla návrh zákona, kterým se mění některé zákony v oblasti da…",1,"""ps""",2017,20,13,2018-10-23,"""Parlament České republiky, Poslanecká sněmovna 2017 – 2021""","""20. schůze""",true,true,true,null,856
"""Poslanec David Kasal""","""2017ps_stenprot_032schuz_s032060.htm""","""/sqw/detail.sqw?id=6209""","""Děkuji, pane předsedající. Já ke kolegům z pirátské strany. Pokud vypnete proud, a je jedno, jak je vyrobený, tak jsem zvědavý, jak budeme léčit anebo…",7,"""ps""",2017,32,60,2019-06-26,"""Parlament České republiky, Poslanecká sněmovna 2017 – 2021""","""32. schůze""",true,true,true,"""6209""",29
"""Poslanec Miloslav Janulík""","""2021ps_stenprot_071schuz_s071220.htm""","""/sqw/detail.sqw?id=6201""","""A proč ho vlastně nikdo nechce? Jako že by lidi rozhodli jinak, než chceme my? V tom případě jsme ale vedle my, ne oni, to je přece logické, ne? Proto…",1,"""ps""",2021,71,220,2023-07-14,"""PČR, PS 2021-...""","""71. schůze""",true,true,false,"""6201""",1517
"""Poslanec Michal Zuna""","""2021ps_stenprot_098schuz_s098228.htm""","""/sqw/detail.sqw?id=6796""","""Děkuji za slovo, paní předsedající. Já bych se vyjádřil ke dvěma bodům, které jsou součástí této novely takzvaného AML zákona. Oba z nich velmi vítám.…",2,"""ps""",2021,98,228,2024-04-19,"""PČR, PS 2021-...""","""98. schůze""",true,true,true,"""6796""",800
"""Poslanec Jiří Havránek""","""2021ps_stenprot_105schuz_s105055.htm""","""/sqw/detail.sqw?id=6755""","""Děkuji za slovo, kolegyně, kolegové. Mě opravdu těší, že na půdě Parlamentu České republiky se dává prostor projednání návrhu, který se týká kryptoměn…",10,"""ps""",2021,105,55,2024-06-13,"""PČR, PS 2021-...""","""105. schůze""",true,true,true,"""6755""",414
"""Poslanec Patrik Nacher""","""2021ps_stenprot_105schuz_s105055.htm""","""/sqw/detail.sqw?id=6487""","""Děkuji, pane místopředsedo, za slovo. No, ještě k tomu, co říkal kolega Havránek. Jedno první místo tady máme také, na to mě upozornil můj ctěný koleg…",12,"""ps""",2021,105,55,2024-06-13,"""PČR, PS 2021-...""","""105. schůze""",true,true,true,"""6487""",123


In [28]:
porovnani_93_21.filter(pl.col('lemma_right').str.contains('menstr'))

lemma,len_93_21,podil_93_21,lemma_right,len_21_25,podil_21_25,rozdil
str,u32,f64,str,u32,f64,f64
"""menstruační""",2,3.0930e-8,"""menstruační""",77,0.000005,158.849004
"""menstruace""",4,6.1861e-8,"""menstruace""",23,0.000001,23.724202
null,null,null,"""menstruovat""",12,7.6570e-7,null


In [29]:
porovnani_93_21.filter(pl.col('lemma') == 'bydlení')

lemma,len_93_21,podil_93_21,lemma_right,len_21_25,podil_21_25,rozdil
str,u32,f64,str,u32,f64,f64


In [30]:
porovnani_93_21.filter((pl.col('len_93_21') >= 5) & (pl.col('len_21_25') >= 10)).sort(by='rozdil',descending=True).head(150)

lemma,len_93_21,podil_93_21,lemma_right,len_21_25,podil_21_25,rozdil
str,u32,f64,str,u32,f64,f64
"""peštová""",5,7.7326e-8,"""peštová""",1594,0.000102,1315.352268
"""pětikoalice""",13,2.0105e-7,"""pětikoalice""",3969,0.000253,1259.683707
"""Dufek""",6,9.2791e-8,"""Dufek""",378,0.000024,259.934733
"""bek""",8,1.2372e-7,"""bek""",491,0.000031,253.230067
"""havránek""",5,7.7326e-8,"""havránek""",286,0.000018,236.004234
"""f35""",8,1.2372e-7,"""f35""",389,0.000025,200.624229
"""balaš""",5,7.7326e-8,"""balaš""",229,0.000015,188.968425
"""křivoklátsko""",8,1.2372e-7,"""křivoklátsko""",345,0.000022,177.931514
"""hladík""",10,1.5465e-7,"""hladík""",359,0.000023,148.121538


In [31]:
narust = {
    "pětikoalice":"(?i)[^\w]pětikoal",
    "energie, elektřina, plyn":"(?i)[^\w](energ[ie]|plyn|elekt[řr])",
    "korespondenční":"(?i)[^\w]korespondenčn",
#    "omikron":"(?i)omikron",
#    "HHC":"(?i)hhc",
#    "kratom":"(?i)kratom",
#    "drahota":"(?i)drahot",
#    "Green Deal":"(?i)green deal",
    "Ukrajina, Ukrajinci, ukrajinský":"(?i)[^\w]ukrajin",
    "Lex Něco/Někdo":"(?i)[^\w]lex \w{0,20}",
#    "Rusko, ruský":"(?i)[^\w]rusk",
#    "Křivoklát":"(?i)křivoklát",
    "menstruace":"(?i)[^\w]menstru[oau]",
#    "bitcoin":"(?i)bitco",
#    "kryptoměny":"(?i)kryptom",
    "bydlení":"(?i)[^\w](bydle[tn])",
#    "dozimetr":"(?i)dozimet",
    "inflace, zdražování":"(?i)[^\w](infla[cč]|zdraž)"
}

In [32]:
maxima = []
for p, n in narust.items():
    maximum = {
        'slovo':p,
        'max':
        df.filter(
            pl.col('obdobi') == 2021
        ).filter(
            pl.col('text').str.contains(n)
        ).with_columns(
            pl.col('text').str.count_matches(n).alias('pocet_vyskytu')
        ).group_by_dynamic(
            index_column='datum',every='1y'
        ).agg(
            pl.col('pocet_vyskytu').sum()
        ).select(
            pl.col('pocet_vyskytu').max()
        ).item()
    }
    maxima.append(maximum)

maxima = pl.DataFrame(maxima).sort(by='max',descending=True).select(pl.col('slovo')).to_series().to_list()

In [33]:
slov_po_roce = df.group_by_dynamic(index_column='datum',every='1y').agg(pl.col('pocet_slov').sum())
slov_po_ctvrtleti = df.group_by_dynamic(index_column='datum',every='3mo').agg(pl.col('pocet_slov').sum())

In [34]:
def cetnost_po_roce(popisek, hledani):
    return df.filter(
        pl.col('text').str.contains(hledani)
    ).with_columns(
        pl.col('text').str.count_matches(hledani).alias('pocet_vyskytu')
    ).group_by_dynamic(
        index_column='datum',every='1y'
    ).agg(
        pl.col('pocet_vyskytu').sum()
    ).join(
        slov_po_roce, on='datum', how='right'
    ).with_columns(
        (pl.col('pocet_vyskytu') / pl.col('pocet_slov')).alias('podil')
    ).select(
        pl.col(['datum','podil'])
    ).with_columns(
        pl.lit(popisek).alias('co')
    )

In [35]:
def cetnost_po_cvrtleti(popisek, hledani):
    return df.filter(
        pl.col('text').str.contains(hledani)
    ).with_columns(
        pl.col('text').str.count_matches(hledani).alias('pocet_vyskytu')
    ).group_by_dynamic(
        index_column='datum',every='3mo'
    ).agg(
        pl.col('pocet_vyskytu').sum()
    ).join(
        slov_po_ctvrtleti, on='datum', how='right'
    ).with_columns(
        (pl.col('pocet_vyskytu') / pl.col('pocet_slov')).alias('podil')
    ).select(
        pl.col(['datum','podil'])
    ).with_columns(
        pl.lit(popisek).alias('co')
    ).with_columns(
        pl.when(pl.col("datum") >= pl.date(2021,10,1)).then(pl.lit("současná Sněmovna")).otherwise(pl.lit("předchozí Sněmovny")).alias("ktera_snemovna")
    )

In [36]:
do_grafu = pl.DataFrame()
for popis, hled in narust.items():
    do_grafu = pl.concat([do_grafu,cetnost_po_cvrtleti(popis, hled)])

In [37]:
graf = alt.Chart(
    do_grafu.to_pandas(),
    title=alt.Title(
        ["O čem poslanci mluvili častěji než dřív"],
        subtitle=[
            "Údaj v procentech vyjadřuje podíl na celkovém počtu slov. Pozor na rozdílná",
            "měřítka u jednotlivých termínů: čím níže v grafu, tím vzácnější výskyt."
        ]
    ),
    width=330,
    height=50
).mark_bar(
    width=1
).encode(
    alt.X('datum:T', axis=alt.Axis(title=None, tickCount=33, labelAngle=270)),
    alt.Y('podil:Q', axis=alt.Axis(domainOpacity=0, tickColor='#DCDDD6', labelExpr="datum.label * 100 + ' %'",  tickCount=2, orient='right', title=None)),
    alt.Row('co:N', sort=maxima, title=None, header=alt.Header(labelAngle=0, labelAlign='left', labelAnchor='start', labelOrient='top', labelFont='Asap')),
    alt.Color("ktera_snemovna:N", 
              title=None,
              scale=alt.Scale(
                  range=['#6B8EAA','#ba4c43'
                        ]),
              legend=alt.Legend(
                  title=None,
                  direction="horizontal",
                  orient="top"
              )
    )
).configure_axisX(
    grid=False, domain=False
).configure_axisY(
    grid=True, domain=False
).configure_view(
    stroke='transparent'
).resolve_axis(
    x="independent",
    y="independent"
).resolve_scale(
    y="independent"
)

graf

alt.Chart(...)

In [38]:
me_to_neurazi(
    graf=graf, 
    soubor='slova_skokani', 
    kredity='vizualizace: iROZHLAS.cz | zdroj projevů: psp.cz', 
    slozka_na_serveru='grafy-stenoprotokoly'
)

<figure>
    <a href="https://data.irozhlas.cz/grafy-stenoprotokoly/slova_skokani.svg" target="_blank">
    <img src="https://data.irozhlas.cz/grafy-stenoprotokoly/slova_skokani.svg" width="100%" alt="Graf s titulkem „['O čem poslanci mluvili častěji než dřív']“. Další texty by měly být čitelné ze zdrojového souboru SVG." />
    </a>
    </figure>


In [39]:
cetnost_po_roce("pětikoalice","pětikoal")

datum,podil,co
date,f64,str
1993-01-01,null,"""pětikoalice"""
1994-01-01,null,"""pětikoalice"""
1995-01-01,null,"""pětikoalice"""
1996-01-01,null,"""pětikoalice"""
1997-01-01,null,"""pětikoalice"""
1998-01-01,null,"""pětikoalice"""
1999-01-01,null,"""pětikoalice"""
2000-01-01,null,"""pětikoalice"""
2001-01-01,8.1166e-7,"""pětikoalice"""


In [40]:
porovnani_93_21.filter(pl.col("lemma") == "lex")

lemma,len_93_21,podil_93_21,lemma_right,len_21_25,podil_21_25,rozdil
str,u32,f64,str,u32,f64,f64
"""lex""",343,0.000005,"""lex""",864,0.000055,10.393059


In [41]:
porovnani_93_21.filter(pl.col('podil_21_25').is_null()).sort(by='podil_93_21',descending=True).head(150)

lemma,len_93_21,podil_93_21,lemma_right,len_21_25,podil_21_25,rozdil
str,u32,f64,str,u32,f64,f64
null,15020499,0.232294,null,null,null,null
"""votava""",1570,0.000024,null,null,null,null
"""laudát""",1569,0.000024,null,null,null,null
"""popř""",1541,0.000024,null,null,null,null
"""udílím""",1437,0.000022,null,null,null,null
"""Pilip""",1361,0.000021,null,null,null,null
"""payne""",1317,0.00002,null,null,null,null
"""seďa""",1170,0.000018,null,null,null,null
"""dundáčková""",1151,0.000018,null,null,null,null


### O čem se nemluví

In [43]:
porovnani_93_21.filter(~pl.col('podil_93_21').is_null()).filter(pl.col('len_21_25').is_null()).sort(by='len_93_21',descending=True).head(150)

lemma,len_93_21,podil_93_21,lemma_right,len_21_25,podil_21_25,rozdil
str,u32,f64,str,u32,f64,f64
null,15020499,0.232294,null,null,null,null
"""votava""",1570,0.000024,null,null,null,null
"""laudát""",1569,0.000024,null,null,null,null
"""popř""",1541,0.000024,null,null,null,null
"""udílím""",1437,0.000022,null,null,null,null
"""Pilip""",1361,0.000021,null,null,null,null
"""payne""",1317,0.00002,null,null,null,null
"""seďa""",1170,0.000018,null,null,null,null
"""dundáčková""",1151,0.000018,null,null,null,null


In [44]:
porovnani_93_21.filter(~pl.col('podil_93_21').is_null()).filter(~pl.col('len_21_25').is_null()).sort(by='rozdil',descending=False).head(150)

lemma,len_93_21,podil_93_21,lemma_right,len_21_25,podil_21_25,rozdil
str,u32,f64,str,u32,f64,f64
"""tis""",3047,0.000047,"""tis""",1,6.3808e-8,0.001354
"""kováčik""",2753,0.000043,"""kováčik""",1,6.3808e-8,0.001499
"""tzn""",6600,0.000102,"""tzn""",3,1.9143e-7,0.001875
"""jičínský""",2104,0.000033,"""jičínský""",1,6.3808e-8,0.001961
"""mld""",19229,0.000297,"""mld""",11,7.0189e-7,0.00236
"""Wagner""",1679,0.000026,"""Wagner""",1,6.3808e-8,0.002457
"""tejc""",1575,0.000024,"""tejc""",1,6.3808e-8,0.00262
"""stiskněte""",3021,0.000047,"""stiskněte""",2,1.2762e-7,0.002732
"""čnr""",1387,0.000021,"""čnr""",1,6.3808e-8,0.002975
